In [2]:
import sqlite3
import pandas as pd
import psycopg2
import configparser
import jellyfish

ModuleNotFoundError: No module named 'jellyfish'

In [ ]:
try:
    sqliteConnection = sqlite3.connect('DEI Database.db')
    cursor = sqliteConnection.cursor()
    print("Connected to SQLite")
    
    sql_query = """SELECT *
                FROM title_list
                """
    cursor.execute(sql_query)
    sqliteConnection.commit()
    dei_rows = cursor.fetchall()
    cursor.close()
except sqlite3.Error as error:
    print("Failed to run query", error)   

In [ ]:
column_names = ["Genre","Author","Title","Year"]
dei_df = pd.DataFrame(dei_rows, columns=column_names)

In [ ]:
dei_df

In [ ]:
config = configparser.ConfigParser()
config.read('Y:\\SQL Reports\\creds\\app_SIC.ini')

try:
    query = """SELECT
b.best_title AS title,
REPLACE(SPLIT_PART(SPLIT_PART(b.best_author,' (',1),', ',2),'.','')||' '||SPLIT_PART(b.best_author,', ',1) AS author,
SUBSTRING(i.location_code FROM 1 FOR 3) AS location,
CASE
WHEN COUNT(i.id) != 0 THEN TRUE
ELSE FALSE
END AS owns
FROM
sierra_view.bib_record_property b
JOIN
sierra_view.bib_record_item_record_link l
ON
b.bib_record_id = l.bib_record_id
JOIN
sierra_view.item_record i
ON
l.item_record_id = i.id
WHERE
b.material_code = 'a'
AND b.publish_year >= 2018

GROUP BY 1,2,3
ORDER BY 1,2,3
    """
    #variable connection string should be defined in the imported config file
    conn = psycopg2.connect( config['db']['connection_string'] )
except:
    print("unable to connect to the database")
    clear_connection()
cursor = conn.cursor()
cursor.execute(query)
#For now, just storing the data in a variable. We'll use it later.
sierra_rows = cursor.fetchall()
conn.close()
    
column_names_sierra = ["Title", "Author", "Location", "Owns"]
sierra_df = pd.DataFrame(sierra_rows, columns=column_names_sierra)

In [ ]:
sierra_df

In [ ]:
sierra_limited = sierra_df[sierra_df["Owns"] == True]

In [ ]:
sierra_limited

In [ ]:
try:
    sqliteConnection = sqlite3.connect('DEI Database.db')
    cursor = sqliteConnection.cursor()
    print("Connected to SQLite")
    
    sql_query = """SELECT LOWER(REPLACE(Title,'''',''))||LOWER(REPLACE(Author,'''','')) 
                FROM title_list
                """
    cursor.execute(sql_query)
    sqliteConnection.commit()
    title_match = cursor.fetchall()
    title_string = [t[0] for t in title_match]
    cursor.close()
except sqlite3.Error as error:
    print("Failed to run query", error)   

In [ ]:
title_string

In [ ]:
new_title_string = str([str(x).lstrip('\'') for x in title_string]).strip("[]")

In [ ]:
new_title_string

In [ ]:
try:
    query = """\
    SELECT
    b.best_title AS title,
    REPLACE(SPLIT_PART(SPLIT_PART(b.best_author,' (',1),', ',2),'.','')||' '||SPLIT_PART(b.best_author,', ',1) AS author,
    SUBSTRING(i.location_code FROM 1 FOR 3) AS location,
    CASE
    WHEN COUNT(i.id) != 0 THEN TRUE
    ELSE FALSE
    END AS owns
    FROM
    sierra_view.bib_record_property b
    JOIN
    sierra_view.bib_record_item_record_link l
    ON
    b.bib_record_id = l.bib_record_id
    JOIN
    sierra_view.item_record i
    ON
    l.item_record_id = i.id
    WHERE
    b.material_code = 'a'
    AND lower(b.best_title)||LOWER(REPLACE(SPLIT_PART(SPLIT_PART(b.best_author,' (',1),', ',2),'.','')||' '||SPLIT_PART(b.best_author,', ',1)) IN ("""\
    +new_title_string+"""\
    )
    AND b.publish_year >= 2018

    GROUP BY 1,2,3
    ORDER BY 1,2,3
    """
    #variable connection string should be defined in the imported config file
    conn = psycopg2.connect( config['db']['connection_string'] )
except:
    print("unable to connect to the database")
    clear_connection()
cursor = conn.cursor()
cursor.execute(query)
#For now, just storing the data in a variable. We'll use it later.
sierra_rows = cursor.fetchall()
conn.close()

In [ ]:
sierra_rows